# Code for Finding Details of TourRadar Website

In [6]:
import os  
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options 
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import string
   
#fuction for finding lat and longtitude    
def lat_lon(loc):   
    l=[None,None]
    j=loc
    try:
        geolocator = Nominatim(user_agent="specify_your_app_name_here")
        j.translate({ord(c):None for c in string.whitespace})
        location = geolocator.geocode(j,timeout=10000)
        l[0]=location.latitude
        l[1]=location.longitude
        #print(location.address)
    except Exception as e:
        print(e)
    return l
       
def name_add(el):     #function for returning package name
    l=[None,None]
    k=el.find("a",{"class":"ao-clp-algolia-search__anchor ao-clp-algolia-search__anchor--tour-title"})
    if k is not None:
        if k.text is not None:
            l[0]=k.text
        if k.get("href") is not None:
            l[1]=k.get("href")
    return l
def pack_cat(soup):            #returns the category of package
    cat=soup.find("a",{"class":"ao-tour-keep-exploring__element--link"})
    if cat is not None:
        if cat.text is not None:
            return (cat.text)
    else:
        return None
        
def pricee(el):                #returns the price of each package
    p=el.find("dd",{"class":"ao-clp-algolia-search__tour-price-value ao-clp-algolia-search__tour-price-value--from"})
    if p is not None:
        if p.text is not None:
            k=(p.text)[1:]
            ind=k.index(",")
            return (k[:ind]+k[ind+1:])
    else:
        return None
def find_city(el):            #returns the start city
    k=el.find_all("dd",{"class":"ao-clp-algolia-search__definition-value-list aa-text-info"})
    if k is not None:
        p=k[1].text
        ind=p.index("/")
        des=k[0].text
        c=p[:ind]
        return (c+","+des)
    else:
        None
        
def dayy(el):               #returns
    day_no=el.find("dd",{"class":"ao-clp-algolia-search__duration-ppd-info aa-text-info"})
    if day_no is not None:
        if day_no.text is not None:
            return (day_no.text)[:len(day_no.text)-4]
    else:
        return None    
        
def itn(soup):
    t=soup.find("div",{"class":"aa-tour-itinerary__text"})
    if t is not None:
        if t.text is not None:
            return (t.text)[5:len(t.text)-2]
    else:
        return None
def tag_finder(soup):
    st=None
    k=soup.find("ul",{"class":"ao-tour-keep-exploring"})
    if k is not None:
        if k.find_all("li",{"class":"ao-tour-keep-exploring__element"}) is not None:
            tag_find=k.find_all("li",{"class":"ao-tour-keep-exploring__element"})
            for find in tag_find:
                if find.text is not None:
                    if st is not None:
                        st=st+","+(find.text)
                    else:
                        st=(find.text)
    return st
    
head1=["ID","Package Name","Package Category","Nights","Price","State","Overview"]
head2=["ID","Package Id","Day","City","Latitude","Longtitude","Title","Overview","Tag"]   
id_list=[]
itn_list=[]        
package_name_list=[]
days_list=[]
price_list=[]
state_list=[]
package_overview=[]
package_cat_list=[]
address_list=[]
id2=0
chrome_options = Options()  
chrome_options.add_argument("--headless")  
driver =webdriver.Chrome(executable_path=r'C:\Users\DELL\.wdm\drivers\chromedriver\80.0.3987.106\win32\chromedriver.exe')
url="https://www.tourradar.com/search?q="+location+"&hPP=15&idx=tours_production&p=0&dFR%5Bcountries.name%5D%5B0%5D=India"
driver.get(url) 
time.sleep(3)
while True:
    try:
        loadMoreButton =driver.find_element_by_class_name("ais-infinite-hits--showmoreButton")
        time.sleep(5)
        loadMoreButton.click()
        time.sleep(5)
    except Exception as e:
        print(e)
        break
print("Complete")
time.sleep(5)

with open('itinerary_data_ldk.csv', mode='w+',encoding="UTF-8",newline="") as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(head2)


    try:
        soup_catt = BeautifulSoup(driver.page_source, 'html.parser')
        elements=soup_catt.find_all("div",{"class":"ao-clp-algolia-search__tour"})
        id=65
        for el in elements:
            name_add_list=name_add(el)
            packk_name=name_add_list[0]   #fun returning name
            print(name_add_list[0])
            address="https://www.tourradar.com/"+name_add_list[1]   #adding complete address
            address_list.append(address)
            dayyss=dayy(el)
            amount=pricee(el)
           
            state_val=location
            loc=find_city(el)
            print(loc)
            driver.get(address)
            time.sleep(5)
            sub_day=0
            element=driver.find_element_by_class_name("ex--full-width")    #finding expand all tag
            driver.execute_script("arguments[0].click();", element)        #clicking on expand all
            time.sleep(5)
            soup_cat = BeautifulSoup(driver.page_source, 'html.parser') #new pge detail
            time.sleep(2)
            lat_long_list=lat_lon(loc)                                #list returning lat and long values
            lat=lat_long_list[0]                                      #lat at list loc[0]
            long=lat_long_list[1]                                     #long at list loc[1]
            print(lat,long)
            tag=None
            tag_find=tag_finder(soup_cat)                             #function returning tags values
            #soup_cat.find("ul",{"class":"ao-tour-keep-exploring"}) 
            tag=tag_find   
            print(tag)
            s=soup_cat.find("ol",{"class":"det"})      #extracting day wise details 
            if s is not None:                         
                if (s.find_all("li")) is not None:        #day wise details inside li tags
                    day_wise_itn=s.find_all("li")
                    if len(day_wise_itn)>2:
                        
                        for values in day_wise_itn:         #loop to iterate 
                            sub_day=sub_day+1
                            tt=((values.text)).strip()
                            #print(tt)
                            sp_index=tt.find("   ")
                            t=(tt[:sp_index]).strip()
                            if ("Day" in t ):
                                ind_d=t.index("Day")
                                title=(t[ind_d+7:]).strip()
                            if "DAY" in t:
                                ind_d=t.index("DAY")
                                title=(t[ind_d+6:]).strip()
                            overview_text=(tt[sp_index+1:]).strip()
                            id2=id2+1
                            #title=(lines[0]).strip()
                            print("TITLE",title)
                            print("OVERVIEW",overview_text)
                            if len(title)>2:
                                itp_id=id+1
                                writer.writerow([id2,itp_id,sub_day,loc,lat,long,title,overview_text,tag])
                                #package file
                        if len(title)>2:
                            id=id+1
                            itn_list.append(itn(soup_cat))
                            package_cat_list.append(pack_cat(soup_cat))
                            package_name_list.append(packk_name)
                            days_list.append(dayyss)
                            price_list.append(amount)
                            id_list.append(id)
                            state_list.append(location)
                            print(dayyss)
                            print(amount)

        
    except Exception as e:
        print(e)

    finally:
        df=pd.DataFrame(zip(id_list,package_name_list,package_cat_list,days_list,price_list,state_list,itn_list),columns=head1)
        df.to_csv("Packages_data_ldk.csv",index=False,encoding='UTF-8')

    driver.close()

Message: element not interactable
  (Session info: chrome=80.0.3987.122)

Complete
LADAKH BACKPACKING TOUR
Manali,India
32.26309405 77.18812183241408
Sightseeing
Beautiful Ladakh
Leh,India
34.1642029 77.5848133
Adventure
TITLE Arrival At Leh
OVERVIEW In and around LehOn the day of arrival at Leh airport our Holidays At representative will assist you for transfer to your hotel in Leh where you can stay and relax for night and be prepare for next day tour to Sham Valley.   if time and weather permit to visit Leh Palace, Shanti Stupa & Leh Market. Enjoy your evening at own activities. Overnight at Hotel.Start point
 Leh Ladakh Airport , Leh Kushok Bakula Rimpochee Airport (IXL), Ladakh, Ladakh, Jammu and Kashmir, India
TITLE Leh - Sham Valley (75 Kms / 4 Hrs)
OVERVIEW Today morning after breakfast later proceed to downstream along with the Indus River on Leh, en-route visit to Gurudwara Pathar Sahib based in the deep Himalayas.   After 04 Kms visit Magnetic Hills which defies the law of g

28.6141793 77.2022662
None
TITLE Arrive Leh
OVERVIEW In and around LehTravel by air to Leh from Delhi. Meet & greet on arrival and transfer to hotel. Complete day for the acclimatization. In the evening free to make short walk around the Leh bazar Overnight at hotel.
TITLE In Leh
OVERVIEW An excursion to famous monasteries of Ladakh. It is interesting to visit early in the morning to enjoy the praying ceremony at Thiksey monastery. Continue drive to Hemis monastery which is one of the most famous monasteries of the Ladakh region. On the way back visit to Shey Palace. Evening free to explore Leh market on own. Overnight at hotel.
TITLE In Leh
OVERVIEW Morning half day tour visiting Spituk and Phyang monasteries. Afternoon a half day tour visiting Leh Palace, Shanti Stupa, Shankar Gompa and Leh Bazar. Overnight at hotel
TITLE Leh/ Uletokpo (70 kms/ 8 hrs)
OVERVIEW After Breakfast drive to Uletokpo enroute visiting Basgo Palace, Liker and Alchi monasteries. Also walk around the village an

34.1642029 77.5848133
Adventure
TITLE Arrival at Leh
OVERVIEW In and around LehOn the day of arrival at Leh airport Our representative will assist you for transfer to your hotel in Leh where you can stay and relax for night and be prepare for next day tour to Sham Valley. if time and weather permit to visit Leh Palace, Shanti Stupa & Leh Market. Enjoy your evening at own activities. Overnight at Hotel.Start point
 Airport, Railway Station, Hotel, Leh Ladakh, Jammu and Kashmir, India
TITLE Leh - Sham Valley (75 Kms / 4 Hrs)
OVERVIEW Today morning after breakfast later proceed to downstream along with the Indus River on Leh, en-route visit to Gurudwara Pathar Sahib based in the deep Himalayas. After 04 Kms visit Magnetic Hills which defies the law of gravity & further driving through a picturesque landscape we reached the Confluence of The Indus and Zanskar River, just before Saspul a road to the right takes you for your visit to the Likir Monastery, Likir is one of the most active monas

34.1642029 77.5848133
Adventure
TITLE Arrival At Leh
OVERVIEW In and around LehOn the day of arrival at Leh airport our representative will meet and greet you at airport and proceed for your hotel in Leh where you can stay and relax for night and be prepare for next day tour to Sham Valley. If time permits and temperature permits you to visit Leh Palace, Shanti Stupa & Leh Market. Enjoy your evening at own activities. Overnight at Hotel.Start point
 Leh Airport, Leh Kushok Bakula Rimpochee Airport (IXL), Ladakh, Ladakh, Jammu and Kashmir, India
TITLE Leh - Sham Valley (75 Kms / 4 Hrs)
OVERVIEW Today morning after breakfast at hotel in Leh, later proceed to downstream along with the river Indus on Leh, en-route visit to Gurudwara Pathar Sahib based in the deep Himalayas. After 04 Kms visit Magnetic Hills which defies the law of gravity & further driving through a picturesque landscape we reached the Confluence of The Indus and Zanskar River, just before Saspul a road to the right takes 

28.6141793 77.2022662
Adventure,Family,Historical
TITLE Arrival At New Delhi
OVERVIEW In and around New DelhiOn the day of arrival at New Delhi airport, our Holidays At representative will assist you for transfer to your hotel. Relax at the hotel or day free for leisure activities. Overnight at Hotel.Start point
 New Delhi Airport, Terminal 3, Indira Gandhi International Airport, New Delhi, Delhi, India
TITLE At New Delhi
OVERVIEW Today morning after breakfast at hotel, later proceed for full day guided tour to New Delhi and Old Delhi, visit Qutub Minar and drive pass to Lotus Temple also visit Humayun’s Tomb, later drive pass to Rashtrapati Bhavan and India Gate. After lunch at Delhi local city restaurant later proceed for Old Delhi sightseeing tour visit Raj Ghat, Jama Masjid and drive pass to Red Fort also take a short rickshaw ride at Chandni Chowk spice market and huge market for clothes. Overnight at Hotel.
TITLE New Delhi - Agra (220 Kms / 3:30 Hrs)
OVERVIEW Today morning after 

...

34.1642029 77.5848133
None
TITLE Arrive at Leh
OVERVIEW In and around LehThe base to the Stok Kangri summit trek is Leh, that is well accessible by air and by road. Spend the day resting and acclimatizing to the high altitude of Leh. We will pick you from the Airport and drop you to your hotel. *Standard Hotel* . Overnight stay in a hotel. Food is not included as we want you to go out and try the local cuisine.Start point
 Leh Airport, Leh Kushok Bakula Rimpochee Airport (IXL), Ladakh, Ladakh, Jammu and Kashmir, India 07:00 - 10:00
TITLE Rest & Acclimatise in Leh
OVERVIEW Since the trek takes you up to an altitude of 20,180 ft., acclmatisation isn't one thing to be taken gently. Spend the present day exploring in and around Leh. Leh isn't solely a natural surprise however additionally a cultural one. Explore a number of the innumerable ancient monasteries and temples that Leh is applauded for. You’re to select the places you would like to go to. Make most of this present day, and retir

28.6141793 77.2022662
Adventure
Stok Kangri
Leh,India
34.1642029 77.5848133
Hiking & Trekking,Adventure,Mountain Hikes
TITLE Fly into Leh, the capital of Ladakh
OVERVIEW In and around LehSoar over the Greater Himalaya before landing at one of the world’s highest airports. At 10,991 ft. (3,350 m), you will likely feel the altitude already. We transfer to the Kang Lha Chen Hotel and can spend the day at leisure, acclimatizing.
TITLE Exploring the city of Leh (11,480 ft)
OVERVIEW To let your body acclimatize further before the trek we exploring Leh today. You may visit the 11th century monastery at Alchi and the famous Khardung La pass (one of the highest motorable roads in the world). Overnight hotel.
TITLE Trek through Sumdo to Shang Phu (14,250 ft)
OVERVIEW In and around SumdoIn the morning we visit Thiksey Monastery, then continue to Shang. There’s spectacular bird life on today’s three hour trek, including Lammergeyer, Peregrine Falcon, Choughs, Indian Griffon Vulture and the White-w

31.6343083 74.8736788
Discovery
TITLE Join trip in Amritsar
OVERVIEW In and around AmritsarArrive in Amritsar; a city at the spiritual heart of the Sikh religion, its magnificent Golden Temple providing a serene centrepiece to this most holy of cities. 
 
 For those arriving on time our Leader plans to meet you in the hotel reception at 4.30pm for the welcome meeting and to visit the stunning Golden Temple. Shoes must be removed, feet washed and heads covered before we can enter (please bring a headscarf with you or buy one outside). The Golden Temple itself is a surprisingly small building surrounded almost entirely by tanks of holy water in which the Sikh pilgrims bathe. The atmosphere is one of incredible calm with devotees taking their turn to pass through the main shrine where there is a continuous and melodic recital (known as the Akhand Path) from the Guru Granth Sahib, the holy book. We also visit the nearby Jalianwala Gardens, site of the notorious massacre under General Dyer 

31.6343083 74.8736788
Discovery
TITLE Join trip in Amritsar
OVERVIEW In and around AmritsarArrive in Amritsar; a city at the spiritual heart of the Sikh religion, its magnificent Golden Temple providing a serene centrepiece to this most holy of cities. 
 
 For those arriving on time our Leader plans to meet you in the hotel reception at 4.30pm for the welcome meeting and to visit the stunning Golden Temple. Shoes must be removed, feet washed and heads covered before we can enter (please bring a headscarf with you or buy one outside). The Golden Temple itself is a surprisingly small building surrounded almost entirely by tanks of holy water in which the Sikh pilgrims bathe. The atmosphere is one of incredible calm with devotees taking their turn to pass through the main shrine where there is a continuous and melodic recital (known as the Akhand Path) from the Guru Granth Sahib, the holy book. We also visit the nearby Jalianwala Gardens, site of the notorious massacre under General Dyer 

28.6141793 77.2022662
Camping,Adventure,Hiking & Trekking,Historical
TITLE New Delhi
OVERVIEW In and around New DelhiMeeting on arrival and transfer to hotel near the airport.  
Rest of day at leisure (The car and driver at disposal)
TITLE Manali
OVERVIEW In the morning, transfer to the airport to board flight to Manali. 
On arrival at Kullu airport transfer to Manali, check-in to the hotel and relax till afternoon. Have your lunch and then leave for your motorbike ride. Go all the way riding on the Himalayan riding premier through the rough roads along the Kullu Valley. There are tricky bends and stretches on the road which will help you know the torque characteristics of the bike. Enjoy the ride to the beautiful Kullu Valley. Later, come back to the hotel and have a comfortable stay.
TITLE Jispa
OVERVIEW In and around JispaWake up early this morning and then leave for a day full of adventure. So, you start the day with Rothang Pass (3978 meters). This would really turn out to be a ch

In [7]:
data_ldk=pd.read_csv("Packages_data_ldk.csv")
data_ldk.head()

,ID,Package Name,Package Category,Nights,Price,State,Overview
0,66,Beautiful Ladakh,Adventure,8,46788.0,ladakh,Start in Leh and end in Ladakh! With the adve...
1,67,"Monsteries, Lakes and high Passes of Ladakh",10 Best Cultural Travel Companies & Tour Opera...,14,NaN,ladakh,Start and end in New Delhi! With the in-depth...
2,68,Monasteries of Ladakh,10 Best Cultural Travel Companies & Tour Opera...,7,NaN,ladakh,Start and end in New Delhi! With the in-depth...
3,69,Zanskar River Expedition Leh - Ladakh,India from Leh,13,NaN,ladakh,Start and end in Leh! With the white water ra...
4,70,Leh Ladakh Experience,Adventure,9,124928.0,ladakh,Start in Leh and end in Stok! With the advent...


In [8]:
import numpy as np
data_ldk=pd.read_csv("Packages_data_ldk.csv")
rows=data_ldk.shape[0]
#int(np.array(data_ldk.iloc[:,0:1]))
#al=data_ldk.iloc[rows-1:rows,0:1]
val=list(np.array(data_ldk.iloc[:,0:1]))
value=int(val[-1])
print(value)
         

183


In [9]:
import os  
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options 
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import string
   
#fuction for finding lat and longtitude    
def lat_lon(loc):   
    l=[None,None]
    j=loc
    try:
        geolocator = Nominatim(user_agent="specify_your_app_name_here")
        j.translate({ord(c):None for c in string.whitespace})
        location = geolocator.geocode(j,timeout=10000)
        l[0]=location.latitude
        l[1]=location.longitude
        #print(location.address)
    except Exception as e:
        print(e)
    return l
       
def name_add(el):     #function for returning package name
    l=[None,None]
    k=el.find("a",{"class":"ao-clp-algolia-search__anchor ao-clp-algolia-search__anchor--tour-title"})
    if k is not None:
        if k.text is not None:
            l[0]=k.text
        if k.get("href") is not None:
            l[1]=k.get("href")
    return l
def pack_cat(soup):            #returns the category of package
    cat=soup.find("a",{"class":"ao-tour-keep-exploring__element--link"})
    if cat is not None:
        if cat.text is not None:
            return (cat.text)
    else:
        return None
        
def pricee(el):                #returns the price of each package
    p=el.find("dd",{"class":"ao-clp-algolia-search__tour-price-value ao-clp-algolia-search__tour-price-value--from"})
    if p is not None:
        if p.text is not None:
            k=(p.text)[1:]
            ind=k.index(",")
            return (k[:ind]+k[ind+1:])
    else:
        return None
def find_city(el):            #returns the start city
    k=el.find_all("dd",{"class":"ao-clp-algolia-search__definition-value-list aa-text-info"})
    if k is not None:
        p=k[1].text
        ind=p.index("/")
        des=k[0].text
        c=p[:ind]
        return (c+","+des)
    else:
        None
        
def dayy(el):               #returns
    day_no=el.find("dd",{"class":"ao-clp-algolia-search__duration-ppd-info aa-text-info"})
    if day_no is not None:
        if day_no.text is not None:
            return (day_no.text)[:len(day_no.text)-4]
    else:
        return None    
        
def itn(soup):
    t=soup.find("div",{"class":"aa-tour-itinerary__text"})
    if t is not None:
        if t.text is not None:
            return (t.text)[5:len(t.text)-2]
    else:
        return None
    
def change_id(id):
    id=id+1
    return id
def tag_finder(soup):
    st=None
    k=soup.find("ul",{"class":"ao-tour-keep-exploring"})
    if k is not None:
        if k.find_all("li",{"class":"ao-tour-keep-exploring__element"}) is not None:
            tag_find=k.find_all("li",{"class":"ao-tour-keep-exploring__element"})
            for find in tag_find:
                if find.text is not None:
                    if st is not None:
                        st=st+","+(find.text)
                    else:
                        st=(find.text)
    return st
    
head1=["ID","Package Name","Package Category","Nights","Price","State","Overview"]
head2=["ID","Package Id","Day","City","Latitude","Longtitude","Title","Overview","Tag"]   
id_list=[]
itn_list=[]        
package_name_list=[]
days_list=[]
price_list=[]
state_list=[]
package_overview=[]
package_cat_list=[]
address_list=[]
id2=0
chrome_options = Options()  
chrome_options.add_argument("--headless")  
driver =webdriver.Chrome(executable_path=r'C:\Users\DELL\.wdm\drivers\chromedriver\80.0.3987.106\win32\chromedriver.exe')
location="madhya pradesh"
if location=="ladakh":
    url="https://www.tourradar.com/search?q=ladakh&hPP=15&idx=tours_production&p=0&dFR%5Bcountries.name%5D%5B0%5D=India"
else:
    url="https://www.tourradar.com/search?q="+location
driver.get(url) 
time.sleep(3)
while True:
    try:
        loadMoreButton =driver.find_element_by_class_name("ais-infinite-hits--showmoreButton")
        time.sleep(5)
        loadMoreButton.click()
        time.sleep(5)
    except Exception as e:
        print(e)
        break
print("Complete")
time.sleep(5)

with open('itinerary_data_ldk.csv', mode='a',encoding="UTF-8",newline="") as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(head2)


    try:
        soup_catt = BeautifulSoup(driver.page_source, 'html.parser')
        elements=soup_catt.find_all("div",{"class":"ao-clp-algolia-search__tour"})
        id=value
        for el in elements:
            name_add_list=name_add(el)
            packk_name=name_add_list[0]   #fun returning name
            print(name_add_list[0])
            address="https://www.tourradar.com/"+name_add_list[1]   #adding complete address
            address_list.append(address)
            dayyss=dayy(el)
            amount=pricee(el)
           
            state_val=location
            loc=find_city(el)
            print(loc)
            driver.get(address)
            time.sleep(5)
            sub_day=0
            element=driver.find_element_by_class_name("ex--full-width")    #finding expand all tag
            driver.execute_script("arguments[0].click();", element)        #clicking on expand all
            time.sleep(5)
            soup_cat = BeautifulSoup(driver.page_source, 'html.parser') #new pge detail
            time.sleep(2)
            lat_long_list=lat_lon(loc)                                #list returning lat and long values
            lat=lat_long_list[0]                                      #lat at list loc[0]
            long=lat_long_list[1]                                     #long at list loc[1]
            print(lat,long)
            tag=None
            tag_find=tag_finder(soup_cat)                             #function returning tags values
            #soup_cat.find("ul",{"class":"ao-tour-keep-exploring"}) 
            tag=tag_find   
            print(tag)
            s=soup_cat.find("ol",{"class":"det"})      #extracting day wise details 
            if s is not None:                         
                if (s.find_all("li")) is not None:        #day wise details inside li tags
                    day_wise_itn=s.find_all("li")
                    if len(day_wise_itn)>2:
                        
                        for values in day_wise_itn:         #loop to iterate 
                            sub_day=sub_day+1
                            tt=((values.text)).strip()
                            #print(tt)
                            sp_index=tt.find("   ")
                            t=(tt[:sp_index]).strip()
                            if ("Day" in t ):
                                ind_d=t.index("Day")
                                title=(t[ind_d+7:]).strip()
                            if "DAY" in t:
                                ind_d=t.index("DAY")
                                title=(t[ind_d+6:]).strip()
                            overview_text=(tt[sp_index+2:]).strip()
                            id2=id2+1
                            #title=(lines[0]).strip()
                            print("TITLE",title)
                            print("OVERVIEW",overview_text)
                            if len(title)>2:
                                itp_id=change_id(id)
                                writer.writerow([id2,itp_id,sub_day,loc,lat,long,title,overview_text,tag])
                                #package file
                        if len(title)>2:
                            id=id+1
                            itn_list.append(itn(soup_cat))
                            package_cat_list.append(pack_cat(soup_cat))
                            package_name_list.append(packk_name)
                            days_list.append(dayyss)
                            price_list.append(amount)
                            id_list.append(id)
                            state_list.append(location)
                            print(dayyss)
                            print(amount)

        
    except Exception as e:
        print(e)

    finally:
        df=pd.DataFrame(zip(id_list,package_name_list,package_cat_list,days_list,price_list,state_list,itn_list),columns=head1)
        df.to_csv("Packages_data_ldk_df2.csv",index=False,encoding='UTF-8')

    driver.close()

Message: no such element: Unable to locate element: {"method":"css selector","selector":".ais-infinite-hits--showmoreButton"}
  (Session info: chrome=80.0.3987.122)

Complete
Trip to Madhya Pradesh Wildlife Safari
Jabalpur,India
23.1608938 79.9497702
Safari,Family,Wildlife,Christmas & New Year
TITLE Arrival at Jabalpur - Bandhavgarh (180 Kms / 3 Hrs)
OVERVIEW In and around BandhavgarhOn the day of arrival at Jabalpur airport, our Yatra Exotic representative will assist you at airport and later drive to Bandhavgarh. Upon arrival at Bandhavgarh, check-in to your hotel for night stay. Overnight at Hotel.Start point
 Jabalpur, Jabalpur Rly Station, Jabalpur, Madhya Pradesh, India
TITLE At Bandhavgarh
OVERVIEW Early morning before breakfast drive to Bandhavgarh Tiger Reserve - This biodiverse park is known for its large population of royal Bengal tigers and other animals include white tigers, leopards and deer. The mix of tropical forest, Sal trees and grassland is home to scores of bird sp

28.6141793 77.2022662
Wildlife,Tiger,Christmas & New Year
TITLE Arrive at Delhi
OVERVIEW Arrive at the Delhi airport; After the customs formalities, meet our MTA representative and get transferred to the hotel for an overnight stay.Start point
 delhi, Delhi, India 16:00 - 23:55
TITLE Delhi - Nagpur – Kanha
OVERVIEW After breakfast, in time transfer to the airport to board a flight to Nagpur. Upon arrival, drive to Kanha National Park (260 km). Upon arrival in Kanha, check-in at the lodge. Overnight stay
TITLE Kanha
OVERVIEW Explore wildlife in the park through morning and evening jungle safari. Overnight in the lodge.
TITLE Kanha – Bandhavgarh national park
OVERVIEW Enjoy morning safari in Kanha. After breakfast, proceed to Bandhavgarh national park by road. On arrival, check-in at the lodge for overnight stay.
TITLE Bandhavgarh
OVERVIEW Enjoy morning and evening safari into the Bandhavgarh national park. Explore the wildlife of the region. Try to spot the big cats. Overnight stay at t

28.6141793 77.2022662
Yoga Retreats,Christmas & New Year
TITLE New Delhi Arrival
OVERVIEW In and around New DelhiUpon arrival in Delhi, you will be met with one of our representatives. You will then be escorted to the hotel where you will be checked in. You will have the rest of the day to yourself for leisure, so you can either relax, or explore the neighborhood. Delhi, or the National Capital Territory of Delhi, has been the capital since 1911, when there still was the British Raj. Apart from that, Delhi has served as the capital of several empires, and each of the rulers has left a mark here. From Jantar Mantar and Red Fort by Shah Jehan, to Humanyu’s Tomb, to Qutub Minar by Qutub ud din Aibak and Iltutmish, Delhi has a range of places with connections to the royal times. Moreover, there are various other famous places, in both New Delhi, and Old Delhi, which one can’t simply miss. Overnight stay will be at the hotel in Delhi.Start point
 We pick up uests from New Delhi airport or a

21.1498134 79.0820556
Adventure,Christmas & New Year
TITLE Reporting In Nagpur, Transfer To Tadoba National Park; Overnight In Resort
OVERVIEW Upon your arrival in Nagpur airport or railway station, a warm welcome on spot by our representative and transfer to your wildlife resort in Tadoba by a luxury AC car. The journey will be of 3 and half hours and after dropping at resort, complete check in procedure and rest for a while. At lunch time, savour in the delights of multi-cuisine lunch and then, spend time in leisure. In the evening, have a coffee or tea and then walk around the resort for nature walk. At night, a delectable dinner and then leisurely overnight stay in resort will wind up the day. Start point
 We Pick up from all Hotels/Airport in Nagpur, Nagpur, Maharashtra, India 23:55 - 01:05
TITLE Jungle Safari In Tadoba National Park; Overnight In Resort
OVERVIEW Early morning safari arrangement to deep forest area of Tadoba National Park is arranged for you. For that, our jeep wi

28.6141793 77.2022662
Adventure,Christmas & New Year
TITLE Arrival In Delhi; Pick Up & Transfer To Hotel
OVERVIEW At Delhi airport , receive a warm welcome from one of our representative and get smoothly transferred to a hotel where your accommodation is pre-booked by us. By providing ID proof and other details, complete check in formalities and then get stay there. After freshen up, savour in the delectable lunch or refreshments and then spend leisure time in hotel.  After lunch, you will be provided assistance for a comfortable and hassle-free Delhi sightseeing. In this sightseeing, you will sightsee attractions of Old Delhi as well as New Delhi. So, the key tourists’ sites of Delhi sightseeing include Jama Masjid, QutubMinar, Red Fort, India Gate, Raj Ghat, Parliament House, President House and a ride in ChandaniChowk for shopping.  Overnight stay and dinner is arranged at hotel itself.  Start point
 We pick up from all hotels/airport in New Delhi/NCR, New Delhi, Delhi, India 23:55 

21.1498134 79.0820556
Adventure,Christmas & New Year
TITLE Reaching Nagpur, Transfer To Hotel; Overnight Stay
OVERVIEW Your India’s Wildlife Treasures Tour services start with your arrival in Nagpur. Based on your arrival, our representative approaches you and offer a warm welcome on spot. After welcome, transfer to hotel for resting and overnight stay. Start point
 We Pick up our visitors from Airport/Hotel in Nagpur, Nagpur, Maharashtra, India 01:05 - 23:55
TITLE Transfer To Pench Tiger Reserve; Overnight In Resort
OVERVIEW Next morning, you will be moved to Pench National Park in a luxury AC car organised by us. Before this journey, feed yourself with a delighting breakfast in hotel and then check out. On reaching Pench, first get your accommodation in a pre-booked wildlife resort. Spend quality hours in rejuvenation at resort. Enjoy lunch and evening snacks and go on the nature walk around resort. Overnight in resort.
TITLE Safaris In Pench Reserve; Overnight In Resort
OVERVIEW To 

'NoneType' object has no attribute 'latitude'
None None
Safari,Adventure,Wildlife
TITLE Delhi
OVERVIEW Welcome to Delhi – the capital of wondrous India. Your wilderness and wildlife adventure will begin with an important welcome meeting at 6 pm, where you’ll meet your group leader and fellow travellers. As this trip spends very little time in Delhi, we recommend you spend arrive a few extra days before your trip if you want to experience all the city has to offer. There’s Jama Masjid, Humayun's Tomb, Qutub Minar, the Red Fort, and so much more. If you would like to spend more time here, we’ll be happy to organise additional accommodation (subject to availability). Accommodation: Hotel
TITLE Bhopal
OVERVIEW In and around BhopalToday, head to the airport to take a flight to Bhopal (approximately 1.5 hours, not including time spent at the airport). Bhopal is the capital of the central state of Madhya Pradesh, and is flanked by two large lakes, the Upper and Lower Lake. It is one of India’

28.6141793 77.2022662
In-depth Cultural,Family,Christmas & New Year
TITLE - Arrive Delhi
OVERVIEW Arrive in Delhi. Delhi is one of the oldest inhabited cities in the world, Welcome at Indira Gandhi International Airport and transfer to hotel.Later we shall start sightseeing tour with Jama Masjid, this is India’s largest mosque that can incredibly embrace 25,000 people. The ‘Friday Mosque’ was Shah Jahan’s last architectural creation, built between 1644 and 1658. The Jama Masjid is covered with elaborate carvings and has stanzas etched from the holy Quran.Enjoy a rickshaw ride through the winding streets of Chandni Chowk.The impressive sandstone structure Red Fort. It was the seat of the Mughal emperor Shah Jahan till two centuries. This iconic symbol of India is used as a stage by the Prime Minister of India to address the public and hoist the National flag on Independence Day.Later drive past through imposing India Gate. India Gate is a red sandstone Arch and is also known as All Indi

28.6141793 77.2022662
In-depth Cultural
TITLE Arrive Delhi
OVERVIEW Arrive in Delhi. On arrival, you will be welcomed by our representative at International Airport and will be transferred to the hotel (Rooms will be available from 1400 Hrs)
TITLE Delhi-Indore (By flight)
OVERVIEW After breakfast transfer to the airport to board the flight for Indore.  DEL/IDR	6E 733	 	ETD/ETA	1105/1230 Hrs  On arrival, transfer to hotel for check in.  Later enjoy the sightseeing tour of the city visiting the Rajwada or the Old Palace (a major centre of attraction), Lalbagh palace, Kanch Mandir (a marvellous Jain temple), Chattri Bagh (the memorial with some canopies) and the Bada Ganapati (the largest lord Ganesha statue in the world).
TITLE Indore-Mandu (By surface 95 Kms/ 02 Hrs)
OVERVIEW After early breakfast, drive to Mandu. On arrival, check-in to the hotel.  It is a dream city steeped in legends of the love of Baz Bahadur for his beloved Rani Roopmati. Breathtakingly beautiful, this former capit

21.2379469 81.6336833
Wildlife,Christmas & New Year
TITLE Arrival in Kanha National Park, Evening Nature Walk
OVERVIEW Welcome to India ! You are met by our Dizkvr Experience Officer at the at Raipur airport and set off for Kanha National Park. Witness the most scenic drive as you pass through traditional tribal villages, reserve forests and mountain roads till you reach to your luxurious yet eco -sensitive stay for the next 3 days. Kanha Earth lodge architecture has been inspired by the Gond tribal villages situated on the fringes of Kanha. The lodge has been built with high standards towards eco sensitivity, local village cooperation and a great wilderness experience. As a conscious decision the lodge is situated remotely, away from the tourism hub, yet close enough for an easy access to the park. As you arrive at Kanha Earth Lodge, recharge yourself with a delicious lunch in the indoor setting. Set yourself up for an evening walk in Mandla Village for interaction and a school visit.

In [10]:
df_mp=df
bigdata = data_ldk.append(df_mp, ignore_index=True)
bigdata.to_csv("package_ldk_mp.csv",index=False,encoding='UTF-8')
bigdata.head()

,ID,Package Name,Package Category,Nights,Price,State,Overview
0,66,Beautiful Ladakh,Adventure,8,46788,ladakh,Start in Leh and end in Ladakh! With the adve...
1,67,"Monsteries, Lakes and high Passes of Ladakh",10 Best Cultural Travel Companies & Tour Opera...,14,NaN,ladakh,Start and end in New Delhi! With the in-depth...
2,68,Monasteries of Ladakh,10 Best Cultural Travel Companies & Tour Opera...,7,NaN,ladakh,Start and end in New Delhi! With the in-depth...
3,69,Zanskar River Expedition Leh - Ladakh,India from Leh,13,NaN,ladakh,Start and end in Leh! With the white water ra...
4,70,Leh Ladakh Experience,Adventure,9,124928,ladakh,Start in Leh and end in Stok! With the advent...


In [52]:
new_df=pd.read_csv("itinerary_data_ldk.csv")
new_df1=new_df.iloc[:1507,:]
new_df2=new_df.iloc[1505:,:]
new_df2.head(10)

,ID,Package Id,Day,City,Latitude,Longtitude,Title,Overview,Tag
1505,96,192,6,"Nagpur,India",21.1498134,79.0820556,Safaris In Kanha National Park & Reserve,"Just as the last day, this day will also start...","Adventure,Christmas & New Year"
1506,97,192,7,"Nagpur,India",21.1498134,79.0820556,Moving To Bandhavgarh National Park; Overnight...,"Now, you are about to leave Kanha National Par...","Adventure,Christmas & New Year"
1507,98,192,8,"Nagpur,India",21.1498134,79.0820556,Safari Excursions In Bandhavgarh National Park...,Enjoy morning safari excursion to discover the...,"Adventure,Christmas & New Year"
1508,99,192,9,"Nagpur,India",21.1498134,79.0820556,Moving To Satpura National Park Via Jabalpur &...,Early morning by road journey to Jabalpur in a...,"Adventure,Christmas & New Year"
1509,100,192,10,"Nagpur,India",21.1498134,79.0820556,Safari Experiences In Satpura; Overnight In Hotel,"Next morning, proceed on jungle safari in an o...","Adventure,Christmas & New Year"
1510,101,192,11,"Nagpur,India",21.1498134,79.0820556,Morning & Afternoon Safaris In Satpura Nationa...,The tour plan for this day is almost similar a...,"Adventure,Christmas & New Year"
1511,102,192,12,"Nagpur,India",21.1498134,79.0820556,Morning Safari In Satpura Reserve; Moving To B...,"In and around BhopalNext morning, your last wi...","Adventure,Christmas & New Year"
1512,103,192,13,"Nagpur,India",21.1498134,79.0820556,Departure Day,This is the last and the departure day from ou...,"Adventure,Christmas & New Year"
1513,104,193,1,"Delhi,India",28.6517178,77.2219388,Arrive Delhi,On arrival at Delhi airport assistance & trans...,"Sightseeing,Christmas & New Year"
1514,105,193,2,"Delhi,India",28.6517178,77.2219388,Delhi - Bhopal (flight),"In and around Bhopal Breakfast at hotel, tran...","Sightseeing,Christmas & New Year"


In [23]:
import os  
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options 
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import string
   
#fuction for finding lat and longtitude    
def lat_lon(loc):   
    l=[None,None]
    j=loc
    try:
        geolocator = Nominatim(user_agent="specify_your_app_name_here")
        j.translate({ord(c):None for c in string.whitespace})
        location = geolocator.geocode(j,timeout=10000)
        l[0]=location.latitude
        l[1]=location.longitude
        #print(location.address)
    except Exception as e:
        print(e)
    return l
       
def name_add(el):     #function for returning package name
    l=[None,None]
    k=el.find("a",{"class":"ao-clp-algolia-search__anchor ao-clp-algolia-search__anchor--tour-title"})
    if k is not None:
        if k.text is not None:
            l[0]=k.text
        if k.get("href") is not None:
            l[1]=k.get("href")
    return l
def pack_cat(soup):            #returns the category of package
    cat=soup.find("a",{"class":"ao-tour-keep-exploring__element--link"})
    if cat is not None:
        if cat.text is not None:
            return (cat.text)
    else:
        return None
        
def pricee(el):                #returns the price of each package
    p=el.find("dd",{"class":"ao-clp-algolia-search__tour-price-value ao-clp-algolia-search__tour-price-value--from"})
    if p is not None:
        if p.text is not None:
            k=(p.text)[1:]
            ind=k.index(",")
            return (k[:ind]+k[ind+1:])
    else:
        return None
def find_city(el):            #returns the start city
    k=el.find_all("dd",{"class":"ao-clp-algolia-search__definition-value-list aa-text-info"})
    if k is not None:
        p=k[1].text
        ind=p.index("/")
        des=k[0].text
        c=p[:ind]
        return (c+","+des)
    else:
        None
        
def dayy(el):               #returns
    day_no=el.find("dd",{"class":"ao-clp-algolia-search__duration-ppd-info aa-text-info"})
    if day_no is not None:
        if day_no.text is not None:
            return (day_no.text)[:len(day_no.text)-4]
    else:
        return None    
        
def itn(soup):
    t=soup.find("div",{"class":"aa-tour-itinerary__text"})
    if t is not None:
        if t.text is not None:
            return (t.text)[5:len(t.text)-2]
    else:
        return None
def tag_finder(soup):
    st=None
    k=soup.find("ul",{"class":"ao-tour-keep-exploring"})
    if k is not None:
        if k.find_all("li",{"class":"ao-tour-keep-exploring__element"}) is not None:
            tag_find=k.find_all("li",{"class":"ao-tour-keep-exploring__element"})
            for find in tag_find:
                if find.text is not None:
                    if st is not None:
                        st=st+","+(find.text)
                    else:
                        st=(find.text)
    return st

def rating_(soup_cat):
    rat=None
    t=soup_cat.find("div",{"class":"ao-tour-sticky-bar__rating-number"})
    if t is not None:
        if t.text is not None:
            rat=(t.text).strip()
    return rat


def get_img(driver):
    img=[None,None,None,None,None]
    #img[0]=driver.find_element_by_xpath('/html/body/main/div[3]/section[1]/div/div[1]/div[1]/img').get_attribute("src")
    i=0
    while i<5:
        driver.find_element_by_xpath('/html/body/main/div[3]/section[1]/div/div[1]/button[2]').click()
        img[i]=driver.find_element_by_xpath('/html/body/main/div[3]/section[1]/div/div[1]/div[1]/img').get_attribute("src")
        i=i+1
        time.sleep(2)
    return img

address_list=[] 
lat_long_list=[]
image_link_list=[]
head1=["Package Id","Package Category","Package Type","Package Name","Nights","Start Price","Min price","Adult Price","Child Price","Overview","Overview short","Inclusions","Exclusions","Package state","About state","is hotel","is transfer","is meals","is sightseeing","Image1","Image2","Image3","Image4","Image5","Discount","Group_var name","package rating ","Review","Reviewer","is active","Date created"]
head2=["ID","Package Id","Day","City","Latitude","Longtitude","Title","Overview","Img1","Img1_line1","Img1_line2","Img2","Img2_line1","Img1_line2","Tag","is_active","date_created"]   


id2=0
chrome_options = Options()  
chrome_options.add_argument("--headless")  
driver =webdriver.Chrome(executable_path=r'C:\Users\DELL\.wdm\drivers\chromedriver\80.0.3987.106\win32\chromedriver.exe')


location="north east india"
url="https://www.tourradar.com/search?q="+location+"&hPP=15&idx=tours_production&p=0&dFR%5Bcountries.name%5D%5B0%5D=India"

driver.get(url) 
time.sleep(10)
while True:
    try:
        loadMoreButton =driver.find_element_by_class_name("ais-infinite-hits--showmoreButton")
        time.sleep(5)
        loadMoreButton.click()
        time.sleep(5)
    except Exception as e:
        print(e)
        break
print("Complete")
time.sleep(5)
# col variable for packages file
package_type="Deluxe"
inclusion="<li>GST on total package</li><li>Accomodation in a well appointed room</li> <li>Meals as per hotel plan and ones mentioned in itinerary</li> <li>All transfers and sightseeing as per the itinerary in an air-conditioned vehicle (except in the hills when the air-conditioner will be switched off)</li> <li>All toll taxes, parking fees and driver allowances</li> <li>All applicable hotel taxes</li>"
exclusion="<li>Anything not mentioned under â€˜Package Inclusionsâ€:tm:</li> <li>All personal expenses, optional tours and extra meals</li> <li>Camera fees, alcoholic/non-alcoholic beverages</li> <li>Vehicle service on leisure days for sightseeing not included in the itinerary</li> <li>Medical and travel insurance</li> <li>Tips, laundry and phone calls</li>"
is_hotel=1
is_transfer=1
is_meals=1
is_sightseeing=1
discount=None
group_var_name=None
min_price=None
adult_price=None
child_price=None
Review=None
Reviewer=None
is_active=1
dt_created="###"
about_state=None

#col variable for itinerary file
is_active_itn=1
dt_created_itn="###"
img1_itn=None
img1_line1=None
img1_line2=None
img2_itn=None
img2_line1=None
img2_line2=None



with open('itinerary_north_east_india.csv', mode='w+',encoding="UTF-8",newline="") as file,open('Packages_north_east_india.csv', mode='w+',encoding="UTF-8",newline="") as pack:
    writer1 = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer1.writerow(head2)
    
    writer2 = csv.writer(pack, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer2.writerow(head1)

    try:
        soup_catt = BeautifulSoup(driver.page_source, 'html.parser')
        elements=soup_catt.find_all("div",{"class":"ao-clp-algolia-search__tour"})
        id=0
        bl=0
        for el in elements:
            name_add_list=name_add(el)
            packk_name=name_add_list[0]   #fun returning name
            print(name_add_list[0])
            address="https://www.tourradar.com/"+name_add_list[1]   #adding complete address
            address_list.append(address)
            dayyss=dayy(el)
            amount=pricee(el)
           
            state_val=location
            loc=find_city(el)    # city name for each day eg nagpur,India
            city_day_wise=loc[:len(loc)-6]
            print(loc)
            print(city_day_wise)
            driver.get(address)
            time.sleep(5)
            sub_day=0
            element=driver.find_element_by_class_name("ex--full-width")    #finding expand all tag
            driver.execute_script("arguments[0].click();", element)        #clicking on expand all
            time.sleep(5)
            image_link_list=get_img(driver)
            img1=image_link_list[0]
            print(img1)
            img2=image_link_list[1]
            img3=image_link_list[2]
            img4=image_link_list[3]
            img5=image_link_list[4]
            soup_cat = BeautifulSoup(driver.page_source, 'html.parser') #new pge detail(inner page)
            time.sleep(2)
            lat_long_list=lat_lon(loc)                                #list returning lat and long values
            lat=lat_long_list[0]                                      #lat at list loc[0]
            long=lat_long_list[1]                                     #long at list loc[1]
            print(lat,long)
            rating=rating_(soup_cat)
            tag=None
            tag_find=tag_finder(soup_cat)                             #function returning tags values
            #soup_cat.find("ul",{"class":"ao-tour-keep-exploring"}) 
            tag=tag_find   
            print(tag)
            s=soup_cat.find("ol",{"class":"det"})      #extracting day wise details 
            if s is not None:                         
                if (s.find_all("li")) is not None:        #day wise details inside li tags
                    day_wise_itn=s.find_all("li")
                    if len(day_wise_itn)>2:
                        
                        for values in day_wise_itn:         #loop to iterate 
                            sub_day=sub_day+1
                            tt=((values.text)).strip()
                            #print(tt)
                            sp_index=tt.find("   ")
                            t=(tt[:sp_index]).strip()
                            if ":" in t[:10]:
                                t=t.replace(":", " ",1)
                            if "'" in t[:10]:
                                t=t.replace("'", " ",1)
                            if "!" in t[:10]:
                                t=t.replace("!", " ",1)
                            if "-" in t[:10]:
                                t=t.replace("-", " ",1)
                            if "Day" in t[:10]:
                                t=t.replace("Day", " ",1)
                            if "DAY" in t[:10]:
                                t=t.replace("DAY", " ",1)
                            if "day" in t[:10]:
                                t=t.replace("day", " ",1)
                            t=t.strip()
                            k=t[:2]
                            if k.isdigit():
                                title=t[3:]
                            else:
                                title=t
                            
                            overview_text=(tt[sp_index+1:]).strip()
                            id2=id2+1
                            #title=(lines[0]).strip()
                           
                            title=title.strip()
                            print("TITLE",title)
                            print("OVERVIEW",overview_text)
                            if len(title)>2:
                                itp_id=id+1
                                writer1.writerow([id2,itp_id,sub_day,city_day_wise,lat,long,title,overview_text,img1_itn,img1_line1,img1_line2,img2_itn,img2_line1,img1_line2,tag,is_active_itn,dt_created_itn])   # for itinerary file
                                #package file
                        image_link_list=get_img(driver)
                        if len(title)>2:
                            id=id+1
                            categ=pack_cat(soup_cat)
                            over=itn(soup_cat)
                            writer2.writerow([id,categ,package_type,packk_name,dayyss,amount,min_price,adult_price,child_price,over,over,inclusion,exclusion,location,about_state,is_hotel,is_transfer,is_meals,is_sightseeing,img1,img2,img3,img4,img5,discount,group_var_name,rating,Review,Reviewer,is_active,dt_created])
                
                            print(dayyss)
                            print(amount)
            bl=bl+1
            print("BLOCK :",bl,"completed")
        
    except Exception as e:
        print(e)

    driver.close()

Message: element not interactable
  (Session info: chrome=80.0.3987.132)

Complete
The Land of Rising Sun - Arunachal (Magical State in North East India)
Guwahati,India
Guwahati
https://cdn.tourradar.com/s3/tour/1500x800/130229_38563d2c.jpg
26.1805978 91.753943
Adventure
TITLE 1  Guwahati - Itanagar
OVERVIEW In and around ItanagarOur team will be waiting for you at the meeting point at 6:30 & Meet all members of the group in the evening followed a Ride Briefing. Our team will help you create the right mindset for your journey. The rental bikes will be allotted there. Please test ride the bike and notify any issue with the team. Later, we will depart from one capital city to another. We would be riding to unheard but the most scenic lands of North East India. We will keep riding along the Bramhaputra river till we reach Itanagar. Overnight leisure at the hotel.Start point
 Guwahati Arrival, Lokpriya Gopinath Bordoloi International Airport, Borjhar, Kamrup, Assam, India 12:00 - 13:00
TIT

15 
110042
BLOCK : 2 completed
8-Night Adventure Tour of Northeast India: Wild Orchids Valleys and Snow-capped Mountains from Guwahati to Jorhat
Guwahati,India
Guwahati
https://cdn.tourradar.com/s3/tour/1500x800/181990_5e1c20f2ef637.jpg
26.1805978 91.753943
Sightseeing,Christmas & New Year
TITLE 1  Guwahati,Assam
OVERVIEW At  7:15am pick up from your hotel in Guwahati and transfer to tour hotel for check in. Breakfast at the hotel. Take a half day tour of the city visiting Kamakhya Devi Temple atop a hotel and the ISKON Temple. Take a walk around Fancy Bazaar and Panbazaar. Lunch at a restaurant. Afterwards take a 4-hour drive to Tezpur. Dinner and overnight. Start point
 Guwahati Airport, Guwahati, Assam, India
TITLE 2  Tezpur- Bomdila- Dirang
OVERVIEW In and around Dirang, Bomdila, TezpurAfter breakfast proceed to Dirang on a four hour journey. Midway is Bomdila from where there would be a steep ascent. The road passes through some of the densest and loveliest forests of India and th

10 
150000
BLOCK : 5 completed
12 Days Assam Wildlife Tour Packages| Wildlife Tour of Assam
Guwahati,India
Guwahati
https://cdn.tourradar.com/s3/tour/1500x800/193676_5e552258dd47d.jpg
26.1805978 91.753943
Adventure,Christmas & New Year
TITLE 1  Arrival In Guwahati, Transfer To Manas National Park; Overnight In Resort
OVERVIEW On your arrival at railway station or airport in Guwahati, you will get a warm welcome and assistance by our chauffer who guides you towards your car for transferring to the famous wildlife destination in Assam – Manas National Park. By road transfer to your pre-booked resort near Manas National Park where complete check in procedure and settle down in your room for resting and meal. Manas National Park is an exotic destination in the state, located in the mesmerising foothills of majestic Himalaya ranges in the North side of River Brahmaputra.  So, during your stay in the resort, you can observe the mesmerising surroundings and hills around and enjoy a relaxing s

12 
166045
BLOCK : 6 completed
Best of North East India in 11 Days through Assam and Nagaland. 
New Delhi,India
New Delhi
https://cdn.tourradar.com/s3/tour/1500x800/168568_2810ec7a.jpg
28.6141793 77.2022662
In-depth Cultural,Christmas & New Year
TITLE 1  Arrival in Guwahati and Transfer to Bhalukpong
OVERVIEW Welcome to India ! Meet our Dizkvr Experience Officer ( DEO ) in Delhi. Transfer to the Guwahati Airport. Drive to the borders of Arunachal Pradesh also called “the land of rising sun”, or orchid state of India, dawn-lit of mountains. Enter into one of the greenest state of the country- Assam. Let us take you to Bhalukpong – famous for its unique natural beauty and is home to hot springs, an orchid centre at Tipi and hosts the Nyethidow festival in March every year. Check into the Wild Masheer and taste the lifestyle of a Colonial tea planter at your stay, a 22-acre sanctuary comprising the jewel in Assam’s crown. Spend your evening at the Tea estate for ‘Chai ki baat’. Surrounded

22 
320800
BLOCK : 8 completed
Assam, Meghalaya & Kaziranga Jungle Safari
Guwahati,India
Guwahati
https://cdn.tourradar.com/s3/tour/1500x800/161975_fef0c3bd.jpg
26.1805978 91.753943
Private,Adventure,Christmas & New Year
TITLE Guwahati
OVERVIEW Our representative will recieve you at the Airport / Railway Station and proceed to your hotel. Enroute, enjoy the magnificient view of the Brahmaputra and check-in at your hotel to refresh. Lunch at an ethnic restaurant. Evening set out to on a River Cruise, to enjoy the beautiful sunset. After the cruise, you will be on a guided tour Exploring the Arts and Crafts, Textile and Silks, etc that is ethnic to the place. O/n at Guwahati.Start point
 Guwahati, Guwahati, Assam, India
TITLE Guwahati to Kaziranga
OVERVIEW Today Morning, set out to see the Kamakhya Temple and a few other Temples associated with It. After Kamakhya Darshan, proceed to Kaziranga National Park. Enroute visit the Kaziranga Orchid Park. Evening check-in to your hotel. We have 

10 
45500
BLOCK : 10 completed
Hornbill Festival Nagaland & Showcasing Northeastern culture
Dimapur,India
Dimapur
https://cdn.tourradar.com/s3/tour/1500x800/173753_4d1648db.jpg
25.9135914 93.7283707
In-depth Cultural
TITLE 1  Arrival at Dimapur – Transfer to Kohima
OVERVIEW In and around DimapurArrive at Dimapur airport, Our representative meets you at the Airport and then get transferred to Kohima . Lunch and dinner will be served inside the hotel . The evening will be spent to explore the interesting night bazaar. Overnight in hotel. NOTE: Nearest Flight connection to Dimapur airport is from Kolkata airport. Usual Kolkata/Dimapur airfare is usd 80. But book it well in advance since closer to festival the fares go very highStart point
 Dimapur airport/hotels, Dimapur, Nagaland, India 08:00 - 11:00
TITLE 2  Hornbill Festival Festivities
OVERVIEW Today we will drive to Kishama heritage village to enjoy the full day at the festival grounds. Later we will visit the interesting traditional

KeyboardInterrupt: 